In [142]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [143]:
books = pd.read_csv('datasets/books-crossing/BX-Books.csv', sep=';', encoding='latin-1',  error_bad_lines=False)
users = pd.read_csv('datasets/books-crossing/BX-Users.csv', sep=';', encoding="latin-1", error_bad_lines=False)
ratings = pd.read_csv('datasets/books-crossing/BX-Book-Ratings.csv', sep=';', encoding="latin-1", error_bad_lines=False)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/Users/mac/opt/anaconda3/envs/recoxplainer/lib/python3.6/site-packages/IPy

In [144]:
books

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [145]:
ratings

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [146]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]
books.rename(columns = {'Book-Title':'title', 'Book-Author':'author', 'Year-Of-Publication':'year', 'Publisher':'publisher'}, inplace=True)
users.rename(columns = {'User-ID':'user_id', 'Location':'location', 'Age':'age'}, inplace=True)
ratings.rename(columns = {'User-ID':'user_id', 'Book-Rating':'rating'}, inplace=True)

/Users/mac/opt/anaconda3/envs/recoxplainer/lib/python3.6/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [147]:
x = ratings['user_id'].value_counts() > 200
y = x[x].index  #user_ids
print(y.shape)
ratings = ratings[ratings['user_id'].isin(y)]

(899,)


In [148]:
rating_with_books = ratings.merge(books, on='ISBN')
rating_with_books

,user_id,ISBN,rating,title,author,year,publisher
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
...,...,...,...,...,...,...,...
487666,275970,1892145022,0,Here Is New York,E. B. White,1999,Little Bookroom
487667,275970,1931868123,0,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA)
487668,275970,3411086211,10,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim"
487669,275970,3829021860,0,The Penis Book,Joseph Cohen,1999,Konemann


In [149]:
number_rating = rating_with_books.groupby('title')['rating'].count().reset_index()
number_rating.rename(columns= {'rating':'number_of_ratings'}, inplace=True)
final_rating = rating_with_books.merge(number_rating, on='title')
final_rating.shape
final_rating = final_rating[final_rating['number_of_ratings'] >= 50]
final_rating.drop_duplicates(['user_id','title'], inplace=True)

In [150]:
final_rating

,user_id,ISBN,rating,title,author,year,publisher,number_of_ratings
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
1,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
2,11676,002542730X,6,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
3,12538,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
4,13552,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,82
...,...,...,...,...,...,...,...,...
236701,255489,0553579983,7,And Then You Die,Iris Johansen,1998,Bantam,50
236702,256407,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50
236703,257204,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50
236704,261829,0553579983,0,And Then You Die,Iris Johansen,1998,Bantam,50


In [151]:
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values="rating")
book_pivot.fillna(0, inplace=True)

In [152]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [153]:
from scipy.sparse import csr_matrix
book_sparse = csr_matrix(book_pivot)

In [154]:
from sklearn.neighbors import NearestNeighbors
model = NearestNeighbors(algorithm='brute')
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [155]:
distances, recommendationsop = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1, -1))

In [156]:
recommendationsop

array([[237, 240, 238, 241, 184]])

In [162]:
book_pivot.index[recommendationsop[0][2]]

'Harry Potter and the Goblet of Fire (Book 4)'

In [164]:
recommendations_df = pd.DataFrame(book_pivot.index[recommendationsop[0]])

In [165]:
recommendations_df

,title
0,Harry Potter and the Chamber of Secrets (Book 2)
1,Harry Potter and the Prisoner of Azkaban (Book 3)
2,Harry Potter and the Goblet of Fire (Book 4)
3,Harry Potter and the Sorcerer's Stone (Book 1)
4,Exclusive


In [166]:
book_pivot.query('index == "New Perspectives: Runes"')

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,


In [167]:
name = "You Belong To Me"
df = book_pivot.loc[book_pivot.index == name]
df

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [168]:
books

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press


In [169]:
books.insert(0, 'book_id', range(1, 1 + len(books)))
books

,book_id,ISBN,title,author,year,publisher
0,1,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,3,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial
3,4,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux
4,5,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company
...,...,...,...,...,...,...
271355,271356,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm)
271356,271357,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books
271357,271358,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco
271358,271359,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press


In [170]:
ratings = ratings.merge(books, on='ISBN')
ratings

,user_id,ISBN,rating,book_id,title,author,year,publisher
0,277427,002542730X,10,3740,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
1,3363,002542730X,0,3740,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
2,11676,002542730X,6,3740,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
3,12538,002542730X,10,3740,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
4,13552,002542730X,0,3740,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc
...,...,...,...,...,...,...,...,...
487666,275970,1892145022,0,53963,Here Is New York,E. B. White,1999,Little Bookroom
487667,275970,1931868123,0,54636,There's a Porcupine in My Outhouse: Misadventu...,Mike Tougias,2002,Capital Books (VA)
487668,275970,3411086211,10,54128,Die Biene.,Sybil GrÃ?Â¤fin SchÃ?Â¶nfeldt,1993,"Bibliographisches Institut, Mannheim"
487669,275970,3829021860,0,53958,The Penis Book,Joseph Cohen,1999,Konemann


In [171]:
ratings = ratings.drop('author', 1).drop('year', 1).drop('publisher', 1)

In [172]:
ratings

,user_id,ISBN,rating,book_id,title
0,277427,002542730X,10,3740,Politically Correct Bedtime Stories: Modern Ta...
1,3363,002542730X,0,3740,Politically Correct Bedtime Stories: Modern Ta...
2,11676,002542730X,6,3740,Politically Correct Bedtime Stories: Modern Ta...
3,12538,002542730X,10,3740,Politically Correct Bedtime Stories: Modern Ta...
4,13552,002542730X,0,3740,Politically Correct Bedtime Stories: Modern Ta...
...,...,...,...,...,...
487666,275970,1892145022,0,53963,Here Is New York
487667,275970,1931868123,0,54636,There's a Porcupine in My Outhouse: Misadventu...
487668,275970,3411086211,10,54128,Die Biene.
487669,275970,3829021860,0,53958,The Penis Book


In [173]:
usersList = ratings.groupby(by='user_id')
users_with_ratings = ratings['user_id'].unique()

In [174]:
#for loop to fill moviesUsedForRecommendations with the movie that will be used for each user to recommend with
booksUsedForRecommendations = []
users_ids = []

for x in range(0, 899):
#    print('x value')
#    print(x)
    allUserRatings = usersList.get_group(users_with_ratings[x])
    allUserRatings = allUserRatings.sort_values(['rating'], ascending=[False])
    book_ids_df = pd.DataFrame(allUserRatings.book_id)
    book_ids_array = np.asarray(book_ids_df)
#    print(book_ids_array)
    
    for y in range(len(book_ids_array)):
#        print('Y value')
#        print(y)
        name_of_id = books.loc[books['book_id'] == book_ids_array[y][0]]
        name_string = name_of_id['title'].item()
        if not (book_pivot.loc[book_pivot.index == name_string]).empty:
#            print('enter if')
            itemUsedForRecommendation = name_string
            users_ids.append(users_with_ratings[x])
            booksUsedForRecommendations.append(itemUsedForRecommendation)
            break  

In [175]:
booksUsedForRecommendations

['Politically Correct Bedtime Stories: Modern Tales for Our Life and Times',
 "Where the Heart Is (Oprah's Book Club (Paperback))",
 'Macgregor Brides (Macgregors)',
 'Politically Correct Bedtime Stories: Modern Tales for Our Life and Times',
 'The Witching Hour (Lives of the Mayfair Witches)',
 'Range of Motion',
 "Bridget Jones's Diary",
 'High Five (A Stephanie Plum Novel)',
 'Cold Mountain : A Novel',
 'The Pillars of the Earth',
 'Fahrenheit 451',
 'Timeline',
 'Skeleton Crew',
 'The Secret Garden',
 'E Is for Evidence: A Kinsey Millhone Mystery (Kinsey Millhone Mysteries (Paperback))',
 'Bag of Bones',
 'Politically Correct Bedtime Stories: Modern Tales for Our Life and Times',
 'Wuthering Heights',
 'A Fine Balance',
 'Year of Wonders',
 'Midnight in the Garden of Good and Evil: A Savannah Story',
 'Anne of Green Gables (Anne of Green Gables Novels (Paperback))',
 'Digital Fortress : A Thriller',
 'The Golden Compass (His Dark Materials, Book 1)',
 'Matilda',
 'Life of Pi',
 'Ha

In [176]:
#loop on moviesUsedForRecommendations and recommend for each user 10 movies & explain
ids = []
recommendations = []
book_names = []
for x in range (len(booksUsedForRecommendations)):
#    print(x)
    userId = x
#    bookId = int(booksUsedForRecommendations[x].book_id)
#    print(movieId)
    bookName = booksUsedForRecommendations[x]
#    print(bookName)
#    print(bookName[0])
#    print(book_pivot.loc[book_pivot.index == bookName[0]])
#    df = book_pivot.loc[book_pivot.index == bookName[0]] 
#    print(book_pivot.loc[book_pivot.index == bookName[0]])
#    if not book_pivot.loc[book_pivot.index == bookName[0]].empty:        
    distances, suggestions = model.kneighbors(book_pivot.loc[book_pivot.index == bookName].values.reshape(1, -1))
#    print(result)
    recommendations_df = pd.DataFrame(suggestions)
#    print(recommendations_df)
    recommendations_array = np.asarray(recommendations_df)
#    print(recommendations_array)
    for i in range (len(recommendations_array)):
        for j in range (len(recommendations_array[i])):
            ids.append(users_ids[x])
            recommendations.append(recommendations_array[i][j])
allUsersRecommendations_df = pd.DataFrame(list(zip(ids, recommendations)),
            columns =['userId', 'book_id'])

In [177]:
allUsersRecommendations_df

,userId,book_id
0,277427,396
1,277427,184
2,277427,308
3,277427,372
4,277427,536
...,...,...
4435,133567,103
4436,133567,632
4437,133567,65
4438,133567,540


In [178]:
books[books['book_id']==396]['title'].item()

'New Perspectives: Runes'

# Explanation

In [179]:
#Computing Assosciation Rules
rules = None
item_sets = [
        [item for item in ratings[ratings.user_id == user].book_id]
        for user in ratings.user_id.unique()
    ]

te = TransactionEncoder()
te_ary = te.fit(item_sets).transform(item_sets)

df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df,
                            min_support=.1,
                            use_colnames=True,
                            max_len=2)

rules = association_rules(frequent_itemsets,
                            metric="lift",
                            min_threshold=.1)
rules = rules[(rules['confidence'] > .1) &
               (rules['lift'] > .1)]

rules.consequents = [list(row.consequents)[0] for _, row in rules.iterrows()]
rules.antecedents = [list(row.antecedents)[0] for _, row in rules.iterrows()]

rules = rules[["consequents", "antecedents", "confidence"]]

In [181]:
usersListAR = ratings.groupby(by='user_id')
pointerAR = 0
explanationsAR = []
for x in range (0,888):
    user_ratingsAR = usersListAR.get_group(users_ids[x]).book_id.values #give it userID
    counterAR = 0
    while counterAR < 5:
        idAR = recommendations[pointerAR]
#        print(idAR)
#        recommendedBookAR = books.loc[books.title == titleAR]
#        print(recommendedBookAR)
#        rec_Origin_id = int(recommendedMovie.origin_iid)
#        rec_item_idAR = recommendedBookAR.book_id
#        print(rec_item_idAR)
        ARs = rules[rules.consequents == idAR]
        #print(ARs)
        explanationAR =  ARs[ARs.antecedents.isin(user_ratingsAR)]
        #print(explanationAR)
        explanationsAR.append({x for x in explanationAR.antecedents})
        pointerAR = pointerAR + 1
        counterAR = counterAR + 1
allUsersRecommendations_df['explanations'] = explanationsAR

In [182]:
allUsersRecommendations_df

,userId,book_id,explanations
0,277427,396,{}
1,277427,184,{}
2,277427,308,{}
3,277427,372,{}
4,277427,536,{}
...,...,...,...
4435,133567,103,{}
4436,133567,632,{}
4437,133567,65,{}
4438,133567,540,{}


In [183]:
expl = allUsersRecommendations_df[[len(x) > 0 for x in allUsersRecommendations_df.explanations]]
fidelity = expl.groupby('userId')['book_id'].count() / 5
modelFidelity = sum(fidelity)/888

In [184]:
modelFidelity

0.0009009009009009009

https://www.analyticsvidhya.com/blog/2021/06/build-book-recommendation-system-unsupervised-learning-project/

In [ ]:
pointerKNN = 0
explanationsKNN = []
for x in range (0,888):
    user_ratingsKNN = usersList.get_group(users_ids[x]).book_id.values #give it userID
    counterKNN = 0
    while counterKNN < 10:
        recBookIdKNN = recommendations[pointerKNN]
        recBookTitleKNN = books[books['book_id']==recBookIdKNN]['title'].item()
        if not (book_pivot.loc[book_pivot.index == recBookTitleKNN]).empty:
            distancesKNN, sim_itemsKNN = model.kneighbors(book_pivot.loc[book_pivot.index == recBookTitleKNN].values.reshape(1, -1))
            explanationKNN =  set(sim_itemsKNN[0]) & set(user_ratingsKNN)
            explanationsKNN.append(explanationKNN)
            pointer = pointer + 1
            counter = counter + 1
        else:
            explanationKNN = {}
            explanationsKNN.append(explanationKNN)
allUsersRecommendations_df['explanations'] = explanations

In [140]:
user_ratingsKNNTest = usersList.get_group(277427).book_id.values #give it userID
recBookIdKNNTest = recommendations[2000]
print(recBookIdKNNTest)
recBookTitleKNNTest = books[books['book_id']==275]['title'].item()
print(recBookTitleKNNTest)
distancesTest, sim_itemsKNNTest = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1, -1))
#sim_itemsKNNTest
#sim_itemsKNNTest[0]
explanationKNNTest =  set(sim_itemsKNNTest[0]) & set(user_ratingsKNNTest)
explanationKNNTest

1
The Golden Compass (His Dark Materials, Book 1)


set()